<a href="https://colab.research.google.com/github/kahram-y/first-repository/blob/master/RAG/recipe-refined_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

✅ 전체 설계 요약 (RAG 최적화 관점)

1️⃣ page_content (임베딩 대상, “의미 검색용”) → 요리 핵심 맥락만 집중

-  요리명: CKG_NM
-  레시피 제목: RCP_TTL
-  요리 소개: CKG_IPDC
-  재료 목록: CKG_MTRL_CN
-  조리 방법/상황/종류: CKG_MTH_ACTO_NM / CKG_STA_ACTO_NM / CKG_KND_ACTO_NM
-  난이도, 시간, 인분: CKG_DODF_NM, CKG_TIME_NM, CKG_INBUN_NM

2️⃣ metadata (필터링·정렬·후처리용) → 검색 후 활용

-  레시피 ID: RCP_SNO
-  조회수 / 추천수 / 스크랩수: INQ_CNT / RCMM_CNT / SRAP_CNT
-  등록자: RGTR_ID
- 카테고리 태그:
-  최초 등록일: FIRST_REG_DT

In [ ]:
# 핵심 라이브러리 통합 설치
!pip install pandas tqdm langchain langchain-community langchain-openai langchain-core chromadb faiss-cpu "huggingface_hub[hf_xet]"

# 모델 및 딥러닝 관련 설치
!pip install -U transformers sentence-transformers torch torchvision torchaudio

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving TB_RECIPE_SEARCH_241226.csv to TB_RECIPE_SEARCH_241226 (2).csv
Saving TB_RECIPE_SEARCH-220701.csv to TB_RECIPE_SEARCH-220701 (2).csv
Saving TB_RECIPE_SEARCH-20231130.csv to TB_RECIPE_SEARCH-20231130 (2).csv


In [ ]:
# 컬럼 매핑 전략
TEXT_COLUMNS = [
    "RCP_TTL", "CKG_NM", "CKG_IPDC", "CKG_MTRL_CN",
    "CKG_MTH_ACTO_NM", "CKG_STA_ACTO_NM",
    "CKG_MTRL_ACTO_NM", "CKG_KND_ACTO_NM",
    "CKG_DODF_NM", "CKG_TIME_NM", "CKG_INBUN_NM"
]

**전처리 + 통합 + RAG용 Document 생성**

In [ ]:
import pandas as pd
from langchain_core.documents import Document
from tqdm import tqdm

# =========================
# 1. CSV 파일 로드 및 통합
# =========================

file_paths = [
    "TB_RECIPE_SEARCH-20231130.csv",
    "TB_RECIPE_SEARCH-220701.csv",
    "TB_RECIPE_SEARCH_241226.csv"
]

dfs = []
for path in file_paths:
    try:
        # 1순위 시도: cp949 + 에러 무시
        df = pd.read_csv(path, encoding="cp949", encoding_errors="ignore")
        print(f"✅ {path} 로드 성공 (cp949)")
    except:
        # 2순위 시도: utf-8
        df = pd.read_csv(path, encoding="utf-8", encoding_errors="ignore")
        print(f"✅ {path} 로드 성공 (utf-8)")
    dfs.append(df)

recipe_df = pd.concat(dfs, ignore_index=True)

# 중복 레시피 제거 (레시피 일련번호 기준)
recipe_df = recipe_df.drop_duplicates(subset=["RCP_SNO"])

print(f"통합 레시피 수: {len(recipe_df)}")

# =========================
# 2. 결측치 처리
# =========================

recipe_df = recipe_df.fillna("정보없음")

# =========================
# 3. page_content 생성 함수
# =========================

def build_page_content(row):
    return f"""
요리명: {row['CKG_NM']}
레시피제목: {row['RCP_TTL']}

[요리 소개]
{row['CKG_IPDC']}

[재료]
{row['CKG_MTRL_CN']}

[요리 정보]
- 요리방법: {row['CKG_MTH_ACTO_NM']}
- 요리상황: {row['CKG_STA_ACTO_NM']}
- 요리종류: {row['CKG_KND_ACTO_NM']}
- 주요재료: {row['CKG_MTRL_ACTO_NM']}
- 난이도: {row['CKG_DODF_NM']}
- 소요시간: {row['CKG_TIME_NM']}
- 인분: {row['CKG_INBUN_NM']}
""".strip()

# =========================
# 4. metadata 생성 함수
# =========================

def build_metadata(row):
    return {
        "recipe_id": row["RCP_SNO"],
        "cook_name": row["CKG_NM"],
        "difficulty": row["CKG_DODF_NM"],
        "time": row["CKG_TIME_NM"],
        "ingredients_tag": row["CKG_MTRL_ACTO_NM"],
        "method": row["CKG_MTH_ACTO_NM"],
        "situation": row["CKG_STA_ACTO_NM"],
        "kind": row["CKG_KND_ACTO_NM"],
        "views": int(row["INQ_CNT"]),
        "likes": int(row["RCMM_CNT"]),
        "scraps": int(row["SRAP_CNT"]),
        "author": row["RGTR_NM"],
        "first_reg_dt": row["FIRST_REG_DT"]
    }

# =========================
# 5. LangChain Document 생성
# =========================

documents = []

for _, row in tqdm(recipe_df.iterrows(), total=len(recipe_df)):
    doc = Document(
        page_content=build_page_content(row),
        metadata=build_metadata(row)
    )
    documents.append(doc)

print(f"RAG용 Document 생성 완료: {len(documents)}")


✅ TB_RECIPE_SEARCH-20231130.csv 로드 성공 (cp949)
✅ TB_RECIPE_SEARCH-220701.csv 로드 성공 (cp949)
✅ TB_RECIPE_SEARCH_241226.csv 로드 성공 (cp949)
통합 레시피 수: 208445


100%|██████████| 208445/208445 [00:21<00:00, 9762.99it/s] 

RAG용 Document 생성 완료: 208445


**임베딩 + Chroma 벡터DB 구축**

In [ ]:
# 임베딩 모델 선택
EMBEDDING_MODEL = "snunlp/KR-SBERT-V40K-klueNLI-augSTS" # 한국어 문장 의미 검색에 매우 강함, 재료/요리 설명 같은 문맥에 적합, OpenAI API 불필요

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# =========================
# 1. 임베딩 모델 로드
# =========================

embedding = HuggingFaceEmbeddings(
    model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS",
    model_kwargs={'device': 'cuda'}, # GPU 사용 설정
    encode_kwargs={"normalize_embeddings": True}
)

/tmp/ipython-input-3041685285.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# =========================
# 2. Chroma DB 생성 (30분소요)
# =========================

persist_directory = "./chroma_recipe_db"

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    persist_directory=persist_directory,
    collection_name="korean_recipe_rag"
)

vectorstore.persist()

print("✅ Chroma 벡터 DB 생성 완료")

벡터DB 로드 (서버 재시작 시)

In [ ]:
# vectorstore = Chroma(
#     persist_directory="./chroma_recipe_db",
#     embedding_function=embedding,
#     collection_name="korean_recipe_rag"
# )

**Retriever 생성**

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 5
    }
)

In [ ]:
# 테스트 검색
query = "두부랑 김치로 간단하게 만들 수 있는 요리"

docs = retriever.invoke(query)

for i, doc in enumerate(docs, 1):
    print(f"\n--- 결과 {i} ---")
    print(doc.page_content[:300])
    print("metadata:", doc.metadata)

**metadata 필터링 검색**

In [ ]:
# Retriever 필터링 예
retriever = vectorstore.as_retriever(
    search_kwargs={
        "k": 5,
        "filter": {
            "difficulty": "초보환영"
        }
    }
)

**품질 향상 로직이 통합된 RAG**
- 조회수 기반 재정렬(Re-ranking)
- 난이도 자동 필터링

In [ ]:
# 조회수 기반 재정렬 및 필터링 함수
def get_refined_context(query, difficulty_filter=None):
    # 1. 기본 검색 (k를 넉넉히 가져옴)
    search_kwargs = {"k": 10}
    if difficulty_filter:
        search_kwargs["filter"] = {"difficulty": difficulty_filter}

    base_docs = vectorstore.similarity_search(query, **search_kwargs)

    # 2. 조회수(views) 기반 재정렬 (내림차순)
    # 검색 유사도와 인기도를 동시에 고려하기 위함
    sorted_docs = sorted(base_docs, key=lambda x: x.metadata.get('views', 0), reverse=True)

    # 3. 상위 5개만 반환
    return sorted_docs[:5]

**GPT 응답 생성용 RAG Chain**

(벡터DB에서 검색한 레시피를 바탕으로 GPT가 “요리 추천 + 조리법 + 재료 대체안”을 자연어로 생성하는 RAG Chain)

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-"

In [ ]:
# 이미 생성한 vectorstore 사용
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

**요리 챗봇 전용 Prompt 설계** (일반 RAG와 차별화되는 부분)

🔹 Prompt 설계 포인트
- 레시피 요약
- 사용자 재료 우선 활용
- 없는 재료는 대체안 제시
- 조리 팁 & 주의사항
- 한국 요리 톤 유지

In [ ]:
# Prompt Template
from langchain_core.prompts import PromptTemplate

RAG_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template="""
당신은 한국 요리에 특화된 요리 추천 AI입니다.

아래는 사용자의 질문과 관련된 실제 레시피 정보입니다.
이 정보를 기반으로 사용자의 상황에 맞게 요리를 추천하고 설명하세요.

====================
[레시피 정보]
{context}
====================

[사용자 질문]
{question}

📌 응답 가이드:
1. 가장 적합한 요리 1~2가지를 추천
2. 각 요리에 대해
   - 요리 개요
   - 핵심 재료
   - 간단한 조리 순서
3. 사용자가 재료가 부족할 경우 가능한 재료 대체안 제시
4. 조리 시간 단축 팁 또는 맛을 살리는 팁 제공
5. 친절하고 자연스러운 한국어로 답변

응답을 시작하세요.
"""
)

LLM (GPT) 설정

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",   # 비용/성능 균형
    temperature=0.4        # 요리는 너무 창의적일 필요 X
)

RAG Chain 구성

In [ ]:
# 방법 1: LCEL(LangChain Expression Language) 방식
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# 앞에서 구축한 vectorstore를 retriever로 변환
retriever = vectorstore.as_retriever()

# 체인 구성: 검색 -> 프롬프트 -> LLM -> 결과 출력 (질문과 검색 결과를 모두 유지)
rag_chain_with_source = RunnableParallel(
    {"context": RunnableLambda(get_refined_context), "question": RunnablePassthrough()}
).assign(
    # 이제 x에는 'context'와 'question' 키가 들어있어 프롬프트와 완벽히 매칭됩니다.
    answer= RAG_PROMPT | llm | StrOutputParser()
)

실행 테스트

In [ ]:
query = "두부랑 김치가 있는데 15분 안에 만들 수 있는 쉬운 한국 요리 추천해줘"
response = rag_chain_with_source.invoke(query)

# 결과 출력
print("="*10, "GPT의 답변", "="*10)
print(response["answer"])

# 참고한 레시피(출처) 확인하기
print("\n" + "="*10, "참고 레시피 목록", "="*10)
for i, doc in enumerate(response["context"], 1):
    print(f"{i}. {doc.metadata.get('cook_name', '알 수 없음')}")
    print(f"난이도: {doc.metadata.get('difficulty', '정보 없음')}, 시간: {doc.metadata.get('time', '정보 없음')}")

재료 기반 질의 강화

In [ ]:
def enrich_query(user_input, ingredients=None):
    if ingredients:
        return f"{user_input}\n\n사용 가능한 재료:\n{', '.join(ingredients)}"
    return user_input

# 1. 사용자 질문 생성
query = enrich_query(
    "간단한 반찬 추천해줘",
    ingredients=["두부", "김치", "양파"]
)

# 2. 체인 실행
try:
    response = rag_chain_with_source.invoke(query)

    print("="*10, "맞춤형 레시피 추천", "="*10)
    print(response["answer"])

    print("\n" + "="*10, "참고한 레시피 출처", "="*10)
    for doc in response["context"]:
        print(f"- {doc.metadata['cook_name']} ({doc.metadata['difficulty']})")

except Exception as e:
    print(f"실행 중 오류 발생: {e}")

**Streamlit 웹 UI**

In [ ]:
pip install streamlit streamlit-extras streamlit-option-menu

In [ ]:
import streamlit as st
from streamlit_extras.add_vertical_space import add_vertical_space
from streamlit_option_menu import option_menu

# =========================
# 페이지 설정
# =========================

st.set_page_config(
    page_title="🥘 냉장고 요리 추천",
    page_icon="🍳",
    layout="wide"
)

# =========================
# 상단 네비게이션
# =========================

selected = option_menu(
    None,
    ["요리 추천", "프로젝트 소개"],
    icons=["egg-fried", "info-circle"],
    orientation="horizontal"
)

# =========================
# 공통 데이터
# =========================

INGREDIENTS = [
    "두부", "김치", "계란", "양파", "대파",
    "감자", "마늘", "고추장", "된장",
    "돼지고기", "닭고기", "소고기", "참치", "어묵"
]

# =========================
# 1️⃣ 요리 추천 페이지
# =========================

if selected == "요리 추천":
    st.title("🥘 냉장고 재료 기반 한국 요리 추천")
    st.caption("만개의레시피 × RAG × GPT")

    add_vertical_space(1)

    col1, col2 = st.columns([1, 2])

    with col1:
        st.subheader("🧺 냉장고 재료")
        selected_ingredients = st.multiselect(
            "보유한 재료 선택",
            INGREDIENTS
        )

        difficulty = st.radio(
            "👩‍🍳 요리 난이도",
            ["상관없음", "초보환영", "보통", "어려움"]
        )

    with col2:
        st.subheader("🍳 만들고 싶은 요리")
        user_query = st.text_area(
            "요청 사항을 입력하세요",
            placeholder="예) 15분 안에 만들 수 있는 간단한 반찬",
            height=120
        )

        if st.button("✨ 요리 추천 받기", use_container_width=True):
            if not user_query:
                st.warning("요청 내용을 입력해주세요!")
            else:
                with st.spinner("👩‍🍳 레시피를 찾고 있어요..."):
                    enriched_query = user_query
                    if selected_ingredients:
                        enriched_query += f"\n\n사용 가능한 재료: {', '.join(selected_ingredients)}"

                    diff = None if difficulty == "상관없음" else difficulty

                    answer, sources = run_rag_with_quality(
                        enriched_query,
                        difficulty=diff
                    )

                st.success("추천 완료!")
                st.markdown("### 🍽 추천 결과")
                st.markdown(answer)

                with st.expander("📚 참고한 레시피"):
                    for d in sources:
                        st.markdown(
                            f"""
**{d.metadata['cook_name']}**
- 난이도: {d.metadata['difficulty']}
- 시간: {d.metadata['time']}
- 조회수: {d.metadata['views']}
"""
                        )

# =========================
# 2️⃣ 프로젝트 소개 페이지
# =========================

if selected == "프로젝트 소개":
    st.title("📌 프로젝트 소개")

    st.markdown("""
### ❓ 해결하는 문제
- 집에 재료는 있는데 **무슨 요리를 할지 모르는 문제**
- 레시피 검색은 키워드 위주 → **재료 활용도가 낮음**
- 초보자는 레시피 선택 자체가 어려움

---

### 🚀 해결 방법
- **RAG 기반 의미 검색**
- 냉장고 재료 + 난이도 + 시간 반영
- 조회수 기반 품질 보정
- GPT를 활용한 자연어 요리 설명

---

### 📊 기대 효과 (가정 기반)
- 요리 결정 시간 **50~70% 단축**
- 식재료 낭비율 **20~30% 감소**
- 요리 실패율 감소 → 사용자 만족도 상승
- 요리 앱 체류 시간 **2배 이상 증가 가능**

---

### 🧾 최종 결과물
- RAG 기반 한국 요리 추천 엔진
- Vector DB (Chroma)
- GPT 응답 생성 파이프라인
- Streamlit 웹 앱 (서비스 데모)
""")